## 연세대 세브란스에서 건강정보에 대한 식단 정보 가져오기

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
import re

url1 = 'https://sev.severance.healthcare/health/lifecare/nutrition/diseasediet.do?&mode=view&articleNo=108731&articleLimit=0'
url2 = 'https://sev.severance.healthcare/health/lifecare/nutrition/diseasediet.do?&mode=view&articleNo=70483&articleLimit=0'

def clean_text(element):
   text = element.get_text().strip()
   text = re.sub(r'\s+', ' ', text)
   return text

class CustomWebLoader(WebBaseLoader):
   def _clean_data(self, soup: bs4.BeautifulSoup) -> str:
       # 자바스크립트 태그 제거
       [s.decompose() for s in soup.find_all('script')]
       
       subject = soup.find('h3', class_='subject')
       title = clean_text(subject) if subject else ""
       
       fr_view = soup.find('div', class_='fr-view')
       if fr_view:
           paragraphs = [p for p in fr_view.find_all('p', recursive=False) 
                        if clean_text(p) and not p.find('img')]
           content = ' '.join(clean_text(p) for p in paragraphs)
           return f"제목: {title}\n\n본문:\n{content}"
       return ""

loader = CustomWebLoader(web_path=(url1, url2))
docs = loader.load()

with open('C:\jupyter\RecipeAI\stroke_diet.txt', 'w', encoding='utf-8') as f:
   f.write(str(docs[0]))

USER_AGENT environment variable not set, consider setting it to identify your requests.
